# Abilities

In [ ]:
#|default_exp game.engine.abilities

In [ ]:
# | hide
%load_ext lab_black

In [ ]:
# | export

from rl18xx.game.engine.core import Ownable, snake_to_pascal

This page contains the different ability types and the attributes
which may be set for each type.

Abilities are mainly used to describe private company powers, but may
also apply to other entities such as corporations. Examples of how
their use can be seen in the [game configuration
directory](../config/game).

## Generic attributes

These attributes may be set for all ability types

- `type`: The name of the ability type
- `owner_type`: The company must be owned by this type of entity in
  order for the ability to be active. Either "player" or
  "corporation".
- `remove`: Game phase when this ability is removed
- `count`: The number of times the ability may be used
- `count_per_or`: The number of times the ability may be used in each OR; the
  property `count_this_or` is reset to 0 at the start of each OR and increments
  each time the ability is used
- `use_across_ors`: If `count` is more than 1 and this is `false`, then the
  ability may only be used within one OR; if an OR starts and the ability has
  been used at least once, but there is still `count` remaining, the ability
  gets used up and removed. Default `true`.
- `on_phase`: The phase when this ability is active
- `after_phase`: The ability is active for all phases after the specified phase
- `when`: (string or array of strings) The game steps or special time descriptor
  when this ability is active. If no values are provided, this ability is
  considered to be "passive", i.e., its effect applies without the user needinng
  to click on the abilities button to activate it. For an ability to be included
  in an `abilities()` call, either a `time` kwarg or the name of the current
  game phase class must match (one of) the ability's `when` string(s). Examples:
    - `any`: usable at any time during the game
    - `buying_train`: train buying step
    - `track`, `track_and_token`: track-laying step; if normal track lays are used
      up, but there is still a `Track` ability, then the active step will not
      pass on to the next step automatically
    - `special_track`: `SpecialTrack` step when it blocks for a private company
      that gets multiple lays
    - `token`: token-placing step
    - `route`: running routes step
    - `sold`: when the company is bought from a player by a corporation
    - `bought_train`: when the owning corporation has bought a train; generally
      used with `close` abilities
    - `owning_corp_or_turn`: usable at any point during the owning corporation's OR turn
    - `owning_player_or_turn`: usable at any point during any of the owning player's OR turns
    - `owning_player_track`: usable during track step during any of the owning player's OR turns
    - `owning_player_sr_turn`: usable at any point during any of the owning player's
    SR turns
    - `or_between_turns`: usable at the start of any corporation's OR turn,
      before that corporation has acted
    - `stock_round`: usable any time during a Stock Round
    - `never`: use with `close` abilities to prevent a company from closing
    - `has_train`: when the owning corporation owns at least one train
    - `operated`: when the owning corporation has finished the dividend step on their first turn

# Base Ability class

In [ ]:
# | export


class AbilityBase(Ownable):
    def __init__(
        self,
        type=None,
        description=None,
        desc_detail=None,
        owner_type=None,
        count=None,
        remove=None,
        use_across_ors=None,
        count_per_or=None,
        passive=None,
        on_phase=None,
        after_phase=None,
        **opts
    ):
        self.type = type
        self.description = description
        self.desc_detail = desc_detail
        self.owner_type = owner_type
        when = opts.get("when", [])
        if not isinstance(when, list):
            when = [when]
        self.when = when
        self.on_phase = on_phase
        self.after_phase = after_phase
        self.count = count
        self.count_per_or = count_per_or
        self.count_this_or = 0
        self.use_across_ors = True if use_across_ors is None else use_across_ors
        self.used = False
        self.remove = remove
        self.start_count = self.count
        self.passive = passive if passive is not None else len(self.when) == 0

    def used(self):
        return self.used

    def use(self, **kwargs):
        self.used = True

        if self.count_per_or:
            self.count_this_or += 1

        if self.count is not None:
            self.count -= 1
            if self.count <= 0:
                self.owner.remove_ability(self)

    def use_up(self):
        while self.count > 0:
            self.use()

    def teardown(self):
        pass

    def when(self, *times):
        return bool(set(self.when) & set(times))

# Ability Classes

## acquire_company

This company comes with a company when acquired.

- `company`: The sym of the additional company to be acquired

In [ ]:
# | export
class AcquireCompany(AbilityBase):
    def __init__(self, company, **kwargs):
        super().__init__(**kwargs)
        self.company = company

## additional_token

Adds 'count' additional tokens to a purchasing company (1817)

In [ ]:
# | export
class AdditionalToken(AbilityBase):
    pass

## assign_corporation

Designate a specific corporation to be the beneficiary of the ability,
for example Steamboat Company in 1846.

When a company with this ability is sold to a corporation, the company is
automatically assigned to the new owning corporation.

- `count`: The number of times the ability may be used
- `closed_when_used_up`: This ability has a count that is decreased each time it is used. If this attribute is true the private is closed when count reaches zero, if false the private
remains open but the discount can no longer be used. Default false.

With this configuration,
the automatic assignment will happen and the company cannot be further
reassigned:

```
{
  "type": "assign_corporation",
  "when": "sold",
  "count": 1,
  "owner_type": "corporation"
}
```

In [ ]:
# | export
class AssignCorporation(AbilityBase):
    def __init__(self, closed_when_used_up, **kwargs):
        super().__init__(**kwargs)
        self.closed_when_used_up = closed_when_used_up

## assign_hexes

Designate a hex to the ability. Usually simulates placement of a
special power token.

- `hexes`: An array of hex coordinates where this ability may be used.

In [ ]:
# | export
class AssignHexes(AbilityBase):
    def __init__(self, hexes, closed_when_used_up=None, cost=0, **kwargs):
        super().__init__(**kwargs)
        self.hexes = hexes
        self.closed_when_used_up = closed_when_used_up
        self.cost = cost

## blocks_hexes

Designate hexes which are blocked by this ability. Use the
`owner_type: "player"` to specify that the blocking ends when the
company is bought in by a corporation.

- `hexes`: An array of hex coordinates that are blocked

In [ ]:
# | export
class BlocksHexes(AbilityBase):
    def __init__(self, hexes, hidden=False, **kwargs):
        super().__init__(**kwargs)
        self.hexes = hexes
        self.hidden = hidden

## blocks_hexes_consent

Designate hexes which are blocked by this ability. Use the `owner_type:
"player"` to specify that the blocking ends when the company is bought in by a
corporation. However unlike `blocks_hexes` this doesn't block the ability except
through a front end confirmation, so players (if they click through) are allowed
to lay a tile on this hex. This is just like when purchasing a train from
another player.

- `hexes`: An array of hex coordinates that are blocked

In [ ]:
# | export
class BlocksHexesConsent(AbilityBase):
    def __init__(self, hexes, hidden=False, **kwargs):
        super().__init__(**kwargs)
        self.hexes = hexes
        self.hidden = hidden

## blocks_partition

Designate a type of partition which this ability disallows crossing.
A partition separates an hex in 2 halves. Use the `owner_type: "player"`
to specify that the blocking ends when the company is bought in by a
corporation.

- `partition_type`: The name of the partition type that is to be
  blocked, akin to terrain and border types.

In [ ]:
# | export
class BlocksPartition(AbilityBase):
    def __init__(self, partition_type, **kwargs):
        super().__init__(**kwargs)
        self.partition_type = partition_type

    def blocks(self, partition_type):
        return self.partition_type == partition_type

## borrow_train

May borrow a train from the Depot for running trains when trainless

- `train_types`: Array of train types that are eligible for borrowing

In [ ]:
# | export
class BorrowTrain(AbilityBase):
    def __init__(self, train_types, **kwargs):
        super().__init__(**kwargs)
        self.train_types = train_types

## choose_ability

Allows you to choose another ability.

- `choices`: The abilities to choose from.

In [ ]:
# | export
class ChooseAbility(AbilityBase):
    def __init__(self, choices=[], **kwargs):
        super().__init__(**kwargs)
        self.choices = choices

## close

Describe when the company closes, using the `when` attribute.

- `corporation'`: If `when` is set to `"train"`, this value is the name
  of the corporation whose train purchase closes this company.

In [ ]:
# | export
class Close(AbilityBase):
    def __init__(self, corporation=None, silent=False, **kwargs):
        super().__init__(**kwargs)
        self.corporation = corporation
        self.silent = silent

## description

Provide a description for an ability that is implemented outside of the ability framework.

- `description`: Description of the ability.

In [ ]:
# | export
class Description(AbilityBase):
    pass

## exchange

This company may be exchanged for a single share of a specified corporation during a step
that allows exchange.

- `corporations`: An array with corporation names, whose share may be exchanged.
  Use a simple `"any"` (no array) to allow for any corporation. Use a simple
  `"ipoed"` (no array) to allow from any company that has been ipoed.
- `from`: Where the share may be take from, either `"ipo"`,
  `"market"`, or an array containing both.

In [ ]:
# | export
class Exchange(AbilityBase):
    def __init__(self, corporations, from_, **kwargs):
        super().__init__(**kwargs)
        self.corporations = corporations
        self.from_ = from_

## generic

Not sure what uses this

In [ ]:
# | export
class Generic(AbilityBase):
    def __init__(self, subtype, from_, **kwargs):
        super().__init__(**kwargs)
        self.type = subtype

## hex_bonus

Give a route bonus if at least one of the hexes are included in the route.

- `hexes`: Name of hexes that gives a bonus.
- `amount`: Revenue bonus.

In [ ]:
# | export
class HexBonus(AbilityBase):
    def __init__(self, hexes, amount, **kwargs):
        super().__init__(**kwargs)
        self.hexes = hexes
        self.amount = amount

## manual_close_company

In [ ]:
# | export
class ManualCloseCompany(AbilityBase):
    pass

## no_buy

This company may not be bought in.

In [ ]:
# | export
class NoBuy(AbilityBase):
    pass

## purchase_train

Immediately purchases the currently available depot train for the owning corporation.
- `free`: If true, the train cost is free, otherwase at cost. Default false.

In [ ]:
# | export
class PurchaseTrain(AbilityBase):
    def __init__(self, free=False, **kwargs):
        super().__init__(**kwargs)
        self.free = free

## reservation

Reserve a token slot

- `hex`: Hex coordinate
- `slot`: A specific token slot to designate
- `city`: Which city to reserve, if multiple cities are on one hex

In [ ]:
# | export
class Reservation(AbilityBase):
    def __init__(self, hex, city=0, slot=0, tile=None, icon=None, **kwargs):
        super().__init__(**kwargs)
        self.hex = hex
        self.city = city
        self.slot = slot
        self.tile = tile
        self.icon = f"/icons/{icon}.svg" if icon else None

    def teardown(self):
        if self.tile:
            self.tile.cities[self.city].remove_reservation(self.owner)

## return_token

Take a station token off the board and place back on the charter
in the most expensive open location

- `reimburse`: If true, the corporation is reimbursed the token cost
  of the location where the token is placed

In [ ]:
# | export
class ReturnToken(AbilityBase):
    def __init__(self, reimburse=False, **kwargs):
        super().__init__(**kwargs)
        self.reimburse = reimburse

## revenue_change

The revenue for this company changes when the conditions set by `when`
and `owner_type` are satisfied.

- `revenue`: The new revenue value

In [ ]:
# | export
class RevenueChange(AbilityBase):
    def __init__(self, revenue, **kwargs):
        super().__init__(**kwargs)
        self.revenue = revenue

## sell_company

This company can be sold to bank for face value. This closes the company.

In [ ]:
# | export
class SellCompany(AbilityBase):
    pass

## shares

This company comes with a share of a corporation when acquired.

- `share`: If a string in the form of `sym_x`, where `sym` is a
  corporation symbol, and `x` is a numeric index, gives the
  certificate of the corporation at index `x` (`x = 0` is the
  president's certificate). If `"random_president"`, gives a
  president's certificate randomly selected at game setup. Gives one
  ordinary share of one the corporations listed in `corporations`,
  randomly selected at game setup.
- `corporations`: A list of corporations to be used with `"share": "random_share"`

In [ ]:
# | export
class Shares(AbilityBase):
    def __init__(self, shares, corporations=None, **kwargs):
        super().__init__(**kwargs)
        self.shares = list(shares) if isinstance(shares, (list, tuple)) else [shares]
        self.corporations = corporations

## teleport

Lay a tile and place a station token without connectivity

- `hexes`: An array of hex coordinates that can be used as the
  teleport destination.
- `tiles`: An array of tile numbers which may be placed at the
  teleport destination.
- `cost`: Cost to use the teleport ability.
- `free_tile_lay`: If true, the tile is laid with 0 cost. Default false.
- `from_owner`: If true, this ability uses a token from the owning corporation's
  charter; if false, an additional token is created. Default true.
- `extra_action`: If true, this ability may be used in addition to the turn's
  normal token placement step. Default false.

In [ ]:
# | export
class Teleport(AbilityBase):
    def __init__(
        self,
        hexes,
        tiles,
        cost=None,
        free_tile_lay=False,
        from_owner=True,
        extra_action=False,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.hexes = hexes
        self.tiles = tiles
        self.cost = cost
        self.free_tile_lay = free_tile_lay
        # If the 'when' list is empty, default it to ['track']
        if not self.when:
            self.when = ["track"]
        self.passive = False
        self.from_owner = from_owner
        self.extra_action = extra_action

## tile_discount

Discount the cost for laying tiles in the specified terrain type

- `discount`: Discount amount
- `terrain`: If set, type of terrain for which discount is provided, otherwise the discount is off the total cost
- `hexes`: If not specified, all applicable hexes qualifies for
  the discount. If specified, only specified hexes qualify
- `exact_match`: Tile may only contain specified terrain type. Default true.

In [ ]:
# | export
class TileDiscount(AbilityBase):
    def __init__(self, discount, terrain=None, hexes=None, exact_match=True, **kwargs):
        super().__init__(**kwargs)
        self.discount = discount
        self.terrain = terrain
        self.hexes = hexes
        self.exact_match = exact_match

    def discounts_tile(self, tile):
        if self.exact_match:
            return tile.terrain == [self.terrain]
        return self.terrain in tile.terrain

## tile_income

Generate extra revenue when tiles are laid on specified terrain types.

- `terrain`: Terrain type for this ability
- `income`: Extra income per tile lay
- `owner_only`: Does this income apply to any tile lay (1882 Tresle Bridge) or just the owner (1817 Mountain Engineers)

In [ ]:
# | export
class TileIncome(AbilityBase):
    def __init__(self, income, terrain=None, owner_only=False, **kwargs):
        super().__init__(**kwargs)
        self.income = income
        self.terrain = terrain
        self.owner_only = owner_only

## tile_lay

Lay or upgrade one or more track tiles without connectivity, in addition to
normal tile lay actions.

- `hexes`: Array of hex coordinates where tiles may be laid.
- `tiles`: Array of tile numbers which may be laid.
- `cost`: Cost to use the ability.
- `closed_when_used_up`: This ability has a count that is decreased each time it is used. If this attribute is true the private is closed when count reaches zero, if false the private
remains open but the discount can no longer be used. Default false.
- `free`: If true, the tiles are laid with 0 cost. Default false.
- `discount`: Discount the cost of laying the tile by the given
  amount. Default 0.
- `special`: If true, do not check that the tile upgrade preserves
  labels and city count. Default true.
- `connect`: If true, and `count` is greater than 1, tiles laid must
  connect to each other. Default true.
- `blocks`: If true and `when` is `sold`, then the step
  `TrackLayWhenCompanySold` will require a tile lay. Default false.
- `reachable`: If true, when tile laid, a check is done if one of the
  controlling corporation's station tokens are reachable; if not a game
  error is triggered. Default false.
- `must_lay_together`: If true and `count` is greater than 1, all the tile lays
  must happen at the same time. If this is `true`, you might want `when` to
  include `special_track`. Default false.
- `must_lay_all`: If true and `count` is greater than 1 and `must_lay_together`
  is true, all the tile lays must be used; if false, then some tile lays may be
  forfeited. Default false.
- `consume_tile_lay`: If true, using this private counts as a corporations tile lay
  and must follow lay/upgrade rules. Upgrade's also count towards the corporations 'upgrade' lays.
  Default false.
- `lay_count` and `upgrade_count` - Use as an alternative to
  `count`. `lay_count` is the number of yellow tile lays, and `upgrade_count` is
  the number of green or higher tile upgrades. When these are set, the ability
  cannot be used for both new tile lays and upgrades. With these set, you need
  to make sure the `ability.use!` call includes an `upgrade` kwarg.

In [ ]:
# | export
class TileLay(AbilityBase):
    def __init__(
        self,
        tiles,
        hexes=None,
        free=False,
        discount=0,
        special=True,
        connect=True,
        blocks=False,
        reachable=False,
        must_lay_together=False,
        cost=0,
        closed_when_used_up=False,
        must_lay_all=False,
        consume_tile_lay=False,
        lay_count=None,
        upgrade_count=None,
        combo_entities=[],
        **kwargs
    ):
        super().__init__(**kwargs)
        self.hexes = hexes
        self.tiles = tiles
        self.free = free
        self.discount = discount
        self.special = special
        self.connect = connect
        self.blocks = blocks
        self.reachable = reachable
        self.must_lay_together = must_lay_together
        self.must_lay_all = must_lay_all and must_lay_together
        self.cost = cost
        self.closed_when_used_up = closed_when_used_up
        self.consume_tile_lay = consume_tile_lay
        self.laid_hexes = []
        self.lay_count = lay_count
        self.upgrade_count = upgrade_count
        if lay_count is not None:
            self.count = lay_count
        self.start_count = self.count
        self.combo_entities = combo_entities

    def use(self, upgrade=False):
        if self.count is not None and self.count <= 0:
            return

        super().use()

        if self.upgrade_count is not None and self.lay_count is not None:
            if upgrade:
                if self.upgrade_count <= 0:
                    raise ValueError("Cannot use this ability to upgrade a tile now")

                self.lay_count = 0
                self.upgrade_count -= 1
                if self.upgrade_count <= 0:
                    self.owner.remove_ability(self)
                    self.count = 0
            else:
                if self.lay_count <= 0:
                    raise ValueError("Cannot use this ability to lay a tile now")

                self.upgrade_count = 0
                self.lay_count -= 1
                if not self.lay_count > 0:
                    self.owner.remove_ability(self)
                    self.count = 0

## token

Modified station token placement

- `hexes`: Array of hex coordinates where this ability may be used
- `city`: Index of the city on the hex where this ability may be used, if
  multiple cities are there
- `price`: Price for placing token
- `teleport_price`: If present, this ability may be used to place a
  token without connectivity, for the given price.
- `discount`: ratio discount from the normal price, e.g., `0.25` takes 25% off
  the token price
- `extra_action`: If true, this ability may be used in addition to the turn's
  normal token placement step. Default false.
- `from_owner`: If true, this ability uses a token from the owning corporation's
  charter; if false, an additional token is created. Default false.
- `cheater`: If an integer is given, this token will be placed into a city at
  whichever is the lowest unoccupied slot index of the following: a regular slot
  in the city; the `cheater` value; one slot higher than the city actually has,
  effectively increasing the city's size by one. (See 18 Los Angeles's optional
  company "Dewey, Cheatham, and Howe" or the corporations which get removed in
  1846 2p Variant for examples). Default nil.
- `extra_slot`: Simlar to `cheater` except this token does not take a slot -
  When `cheater` is used, when the city gets an extra city slot the 'cheater' token
  goes into the newly opened slot. If `extra_slot` is used, when the city gets an extra
  token slot, the new token slot is open - the extra token does not consume it. This
  also means that an `extra_slot` token lay in an city with an open slot does not use
  up the open slot.
- `special_only`: If true, this ability may only be used by explicitly.
  activating the company to which it belongs (i.e., using the `SpecialTrack`
  step); if unset or false, `Engine::Step::Tokener#adjust_token_price_ability!`
  infers that the special ability ought to be used whenever a token is being
  placed in a location that the ability is allowed to use. Default false.
- `neutral`: If true, this ability uses a "neutral" token, which allows all
  corporations to pass through it
- `check_tokenable`: If false, skip the `tokenable?` check before placing the
  token. Used in 18LA2 for the Angeles Public Dump, which places a special
  station token that does not actually belong to the owning corporation, and can
  therefore be placed in the same city as another token belonging to the owning
  corporation. Note that this property will bypass all tokenable checks, not
  just `:existing_token`. Default true.
- `connected`: If true, when token placed, a check is done if the desired token slot
  is connected by track with another city that has a token of the corporation; if not
  a game error is triggered. Default false.

In [ ]:
# | export
class Token(AbilityBase):
    def __init__(
        self,
        hexes,
        price=None,
        teleport_price=None,
        extra_action=False,
        from_owner=False,
        discount=None,
        city=None,
        neutral=False,
        cheater=False,
        extra_slot=False,
        special_only=False,
        check_tokenable=True,
        closed_when_used_up=False,
        connected=False,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.hexes = hexes
        self.price = price
        self.teleport_price = teleport_price
        self.extra_action = extra_action
        self.from_owner = from_owner
        self.discount = discount
        self.city = city
        self.neutral = neutral
        self.cheater = cheater
        self.extra_slot = extra_slot
        self.special_only = special_only
        self.check_tokenable = check_tokenable
        self.closed_when_used_up = closed_when_used_up
        self.connected = connected

    def get_price(self, token=None):
        if token is None or self.discount is None:
            return self.price
        return token.price - (token.price * self.discount)

## train_buy

Modify train buy in some way.

- `face_value`: If true, any inter corporation train buy must be at
  face value. Default false.

In [ ]:
# | export
class TrainBuy(AbilityBase):
    def __init__(self, face_value=None, **kwargs):
        super().__init__(**kwargs)
        self.face_value = bool(face_value)

## train_discount

Discount the train buy cost. The `count` attribute specify how many times the discount can be used.

- `discount`: Discount amount. If > 1 this is an absolute amount. If 0 < amount < 1 it is the fraction, e.g. 0.75 is a 75% discount.
- `trains`: An array of all train names that the discount applies to.
- `closed_when_used_up`: This ability has a count that is decreased each time it is used. If this attribute is true the private is closed when count reaches zero, if false the private
remains open but the discount can no longer be used. Default false.

In [ ]:
# | export
class TrainDiscount(AbilityBase):
    def __init__(self, discount, trains, closed_when_used_up=None, **kwargs):
        super().__init__(**kwargs)
        self.discount = discount
        self.trains = trains
        self.closed_when_used_up = closed_when_used_up

    def discounted_price(self, train, price):
        # If trains are specified and the current train is not in the list, return the original price
        if self.trains and train.name not in self.trains:
            return price

        # Calculate discount value based on whether the discount is a flat value or a percentage
        discount_value = (
            self.discount[train.name]
            if isinstance(self.discount, dict)
            else self.discount
        )

        # Apply the discount to the price
        return price - (
            discount_value if discount_value > 1 else int(price * discount_value)
        )

## train_limit

Modify train limit in some way.
For performance reasons, the supporting code needs to be added directly to the game class. See G18MEX#train_limit for an example.

- `increase`: If positive, this will increase the train limit with this
  amount in all faces. Default 0.

In [ ]:
# | export
class TrainLimit(AbilityBase):
    def __init__(self, increase=None, constant=None, **kwargs):
        super().__init__(**kwargs)
        self.increase = increase
        self.constant = constant

## train_scrapper

In [ ]:
# | export
class TrainScrapper(AbilityBase):
    def __init__(self, scrap_values={}, **kwargs):
        super().__init__(**kwargs)
        self.scrap_values = scrap_values

    def scrap_value(self, train):
        return self.scrap_values.get(train.name, 0)

# Abilities Container

Contains a list of abilities and methods around creating and dealing with them.

In [ ]:
# | export


class Abilities:
    def __init__(self, abilities=[]):
        self._abilities = []

        for ability in abilities:
            if not isinstance(ability, AbilityBase):
                class_name = snake_to_pascal(ability["type"])
                if ability.get("from"):
                    ability["from_"] = ability.pop("from")
                ability_instance = globals()[class_name](**ability)
                ability_instance.owner = self
            else:
                ability_instance = ability
            self._abilities.append(ability_instance)

        self._update_start_counter()

    @property
    def abilities(self):
        return self._abilities

    def set_owner(self, owner):
        self.owner = owner

    def add_ability(self, ability):
        ability.owner = self.owner
        self._abilities.append(ability)
        self._update_start_counter()

    def remove_ability(self, ability):
        ability.teardown()
        self._abilities = [a for a in self._abilities if a != ability]

    def remove_ability_when(self, time):
        for ability in self._abilities[:]:
            if ability.remove == str(time):
                self.remove_ability(ability)

    @property
    def all_abilities(self):
        return self._abilities

    def reset_ability_count_this_or(self):
        for ability in self._abilities:
            ability.count_this_or = 0
            if ability.used and not ability.use_across_ors:
                ability.use_up()

    def ability_uses(self):
        if self._start_count is None:
            return

        count = [0, self._start_count]
        for ability in self._abilities:
            if ability.start_count is not None:
                count = max(
                    count, [ability.count, ability.start_count], key=lambda x: x[1]
                )

        return count

    def _update_start_counter(self):
        start_counts = [
            ability.start_count
            for ability in self._abilities
            if ability.start_count is not None
        ]
        self._start_count = max(start_counts, default=None)

Example usage:

In [ ]:
abilities = Abilities(
    [
        {
            "type": "blocks_hexes",
            "owner_type": "player",
            "hexes": ["I13", "I15"],
        },
        {"type": "close", "when": "bought_train", "corporation": "B&O"},
        {"type": "no_buy"},
        {"type": "shares", "shares": "B&O_0"},
    ]
)

abilities.abilities[3].shares

['B&O_0']